In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# set the max columns to none
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

import statistics
from statistics import mode

import warnings
warnings.filterwarnings('ignore')

# Finalizing Main Dataset

In [6]:
# import dataset hotel, review, and user

# hotel menunggu revisi joane
df_hotel = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataHotel_ML_Filtered.csv', index_col=[0])
df_user = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataUser.csv', index_col=[0])
df_review = pd.read_csv('https://raw.githubusercontent.com/zulfauzi92/Hotel_Recomendation_Model_Traveloka/main/Eksplorasi%20Data/Main%20Dataset/DataReview.csv', index_col=[0])
print(df_hotel.shape)
print(df_user.shape)
print(df_review.shape)

(1311, 268)
(9219, 4)
(9787, 11)


## Hotel Data

In [7]:
# rename some features
df_hotel.rename(columns={'property_id':'hotel_id', 'property_name':'hotel_name'}, inplace=True)

In [8]:
# Non-ML Hotel Feature
hotel_nonML_col = np.append(df_hotel.columns[:8].values,['latitude','longitude','property_type','hotel_facilities','room_facilities'])
df_nonML_hotel = df_hotel[hotel_nonML_col]

# ML Hotel Feature
hotel_ML_col = np.append(['hotel_id','latitude','longitude','hotel_star_rating'], df_hotel.columns[14:].values)
df_ML_hotel = df_hotel[hotel_ML_col]
df_ML_hotel

,hotel_id,latitude,longitude,hotel_star_rating,site_review_rating,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Remote Control TV,Remote Controlled Lights and Fan,Rollaway Beds,Room Door Fitted with Lock and Key,Safe (on request),Sanitary Bin,Second Bathrooms,Separate Bathtubs And Shower,Shared B

In [9]:
# one-hot property type from df_hotel
from sklearn.preprocessing import LabelBinarizer
# one-hot encoding hotel facilities
lb = LabelBinarizer()
x = lb.fit_transform(df_hotel['property_type'])
df_property_type = pd.DataFrame(x, columns=lb.classes_)

# insert one-hotted property type between site_review_rating and hotel facilities
df_ML_hotel1 = df_ML_hotel.iloc[:,:5]
df_ML_hotel2 = df_ML_hotel.iloc[:,5:]

df_ML_hotel3 =  pd.concat([df_ML_hotel1, df_property_type, df_ML_hotel2], axis=1)

### Create Hotel Data for ML and non ML

In [45]:
# Dataframe hotel for ML
print(df_ML_hotel3.shape)
df_ML_hotel3

(1311, 275)


,hotel_id,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Remote Control TV,Remote Controlled Lights an

In [11]:
# update the hotel facilities and room facilities array based on already filtered features
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

# hotel facilities
mlb = MultiLabelBinarizer(classes=df_ML_hotel3.iloc[:, 22:162].columns)
mlb.fit(df_ML_hotel3.iloc[:, 22:162].columns)
df_nonML_hotel['hotel_facilities'] = np.array(mlb.inverse_transform(df_ML_hotel3.iloc[:, 22:162].values), dtype=object)

# # room facilities
mlb2 = MultiLabelBinarizer(classes=df_ML_hotel3.iloc[:, 162:].columns)
mlb2.fit(df_ML_hotel3.iloc[:, 162:].columns)
df_nonML_hotel['room_facilities'] = np.array(mlb2.inverse_transform(df_ML_hotel3.iloc[:, 162:].values), dtype=object)

In [67]:
df_nonML_hotel = pd.concat([df_nonML_hotel.iloc[:,:-2], df_hotel['hotel_star_rating'], df_hotel['site_review_rating'], df_nonML_hotel.iloc[:,-2:]], axis=1)
print(df_nonML_hotel.shape)
df_nonML_hotel.head()

(1311, 15)


,hotel_id,hotel_name,hotel_description,address,point_of_interest,city,state,country,latitude,longitude,property_type,hotel_star_rating,site_review_rating,hotel_facilities,room_facilities
0,H000001,Homewood Suites by Hilton RaleighDurham APRese...,The standard check-in time is 12:00 PM and the...,4603 Central Park Dr,"['Hadimba Temple', 'Naggar Village', 'Himalaya...",Durham,NC,US,35.9,-78.8,Resort,2,4.2,"(Air / Rail Booking, Car Rental, Catering, Chi...","(Attached Bathroom, Bathrobes, Bathroom Amenit..."
1,H000002,Four Points by Sheraton Orlando International ...,The standard check-in time is 12:00 PM and the...,5905 International Drive,"['Sushant Lok', 'Sahara Mall', 'Amity Internat...",Orlando,FL,US,28.5,-81.5,Guest House,0,5.0,"(Baggage Room, Banquet Facilities, Business Ce...","(Air Conditioning, Attached Bathroom, Bathroom..."
2,H000003,Hampton Inn Pittsburgh/West Mifflin,The standard check-in time is 12:00 PM and the...,1550 Lebanon Church Road,"['The Mall', 'Tibetian Monastery', 'Vashisht H...",Pittsburgh,PA,US,40.3,-80.0,Cottage,2,5.0,"(Car Rental, Doctor on Call, Dry Cleaning, Lau...","(Bathroom Amenities, Hot / Cold Running Water,..."
3,H000004,JW Marriott Atlanta Buckhead,The standard check-in time is 12:00 PM and the...,3300 Lenox Road NE,"['International Airport', 'Domestic Airport', ...",Atlanta,GA,US,33.8,-84.4,Hotel,3,4.0,"( Drivers Rest Room, ATM / Banking, Adjoining ...","(Air Conditioning, Bathrobes, Breakfast Availa..."
4,H000005,Best Western-rambler,The standard check-in time is 02:00 PM and the...,457 US Highway 85 87,"['Others', 'K.L. Saini Stadium', 'BAPS Shri Sw...",Walsenburg,CO,US,37.7,-104.8,Homestay,1,3.0,"(Doctor on Call, Front Desk, Lobby, Local Tour...","(Air Conditioning, Attached Bathroom, Ceiling ..."


In [68]:
# save hotel dataset
# df_ML_hotel3.to_csv('Final_Dataset_Hotel_ML.csv') # for ML
# df_nonML_hotel.to_csv('Final_Dataset_Hotel_nonML.csv') # for non ML

## User Data

In [14]:
# create user behavior feature
df_user_final = df_user.copy()
for col in df_ML_hotel3.columns:
    df_user_final[col] = 0

df_user_final['hotel_id'] = np.empty((len(df_user_final), 0)).tolist()

In [15]:
df_review.head(3)

,review_id,user_id,hotel_id,reviews.username,reviews.userCity,reviews.userProvince,name,reviews.rating,reviews.text,reviews.title,reviews.date
0,R000001,U000001,H000021,tatsurok2018,San Jose,UnitedStates,Best Western Plus South Coast Inn,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,2018-01-01T00:00:00.000Z
1,R000002,U000002,H001054,STEPHEN N,San Francisco,CA,Best Western Carmel's Town House Lodge,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,2016-04-02T00:00:00Z
2,R000003,U000003,H001054,15Deborah,Prescott Valley,AZ,Best Western Carmel's Town House Lodge,3,"Parking was horrible, somebody ran into my ren...",Business,2016-01-06T00:00:00Z


In [16]:
df_user_final.head(3)

,user_id,reviews.username,reviews.userCity,reviews.userProvince,hotel_id,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Ra

In [17]:
%%time
# add user behavior value based on user's reviewed hotel
for i,item in df_review.iterrows():
#     melakukan append terhadap array property_id pada df_user_final
    df_user_final.loc[df_user_final['user_id'] == item['user_id'], 'hotel_id'][df_user_final.loc[df_user_final['user_id'] == item['user_id'], 'hotel_id'].index.values[0]].append(item['hotel_id'])
#     melakukan assign fitur hotel facilities dan room facilities
    df_user_final.loc[df_user_final['user_id'] == item['user_id'], df_user_final.columns[7:]] += df_ML_hotel3.loc[df_ML_hotel3['hotel_id'] == item['hotel_id']].iloc[:,3:].values

CPU times: total: 5min
Wall time: 5min 1s


In [18]:
%%time
# updating value for property_id to the most frequent hotel reviewed
most_reviewed = []
for i,item in df_user_final.iterrows():
    most_reviewed.append(mode(df_user_final.loc[i, 'hotel_id']))
# normalize each feature based on the total review by each user
    for col in df_user_final.columns[5:]:
        df_user_final.loc[i, col] = item[col]/len(item['hotel_id'])
df_user_final['most_reviewed'] = most_reviewed

CPU times: total: 1min 52s
Wall time: 1min 52s


In [27]:
%%time
# updating value for latitude and longitude based on the most reviewed hotel
for i,item in df_user_final.iterrows():
    df_user_final.loc[i, 'latitude'] = df_ML_hotel3.loc[df_ML_hotel3['hotel_id'] == item['most_reviewed'], 'latitude'].values
    df_user_final.loc[i, 'longitude'] = df_ML_hotel3.loc[df_ML_hotel3['hotel_id'] == item['most_reviewed'], 'longitude'].values

CPU times: total: 3.86 s
Wall time: 3.87 s


In [28]:
# rename user's location columns
df_user_final.rename(columns={'reviews.username': 'username', 'reviews.userCity':'user_city', 'reviews.userProvince':'user_state'}, inplace=True)
df_user_final.head() 

,user_id,username,user_city,user_state,hotel_id,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Remote 

### Create User Data for ML and non ML

In [48]:
df_nonML_user = df_user_final.iloc[:, :5]
df_nonML_user.rename(columns={'hotel_id':'reviewed_hotel'}, inplace=True)
df_nonML_user

,user_id,username,user_city,user_state,reviewed_hotel
0,U000001,tatsurok2018,San Jose,UnitedStates,[H000021]
1,U000002,STEPHEN N,San Francisco,CA,[H001054]
2,U000003,15Deborah,Prescott Valley,AZ,[H001054]
3,U000004,Wilfredo M,Guaynabo,PR,[H001054]
4,U000005,Luc D,Reno,NV,[H001054]
...,...,...,...,...,...
9214,U009215,Tiffany1017,Wallingford,CT,[H001298]
9215,U009216,bobg187,Homer,AK,[H001298]
9216,U009217,K261ANbrendah,Conway,AR,[H001298]
9217,U009218,soccerrocks2016,Hunter,NewYork,[H001103]


In [46]:
df_ML_user = pd.concat([df_user_final['user_id'], df_user_final['most_reviewed'], df_user_final.iloc[:, 4:-1]], axis=1)
df_ML_user.rename(columns={'hotel_id':'reviewed_hotel'}, inplace=True)
df_ML_user

,user_id,most_reviewed,reviewed_hotel,latitude,longitude,hotel_star_rating,site_review_rating,Beach Hut,BnB,Bungalow,Cottage,Farm Stay,Guest House,Homestay,Hostel,Hotel,Houseboat,Lodge,Motel,Palace,Resort,Service Apartment,Tent,Villa,Bike on Rent,Drivers Rest Room,ATM / Banking,Access Via Exterior Corridors,Adjoining Rooms,Air / Rail Booking,Air-Conditioning - Central,Amphitheatre,Ayurvedic Facilities,Babysitting/ Child Care (surcharge),Badminttion Court,Baggage Room,Ballroom,Banquet Facilities,Bar / Lounge,Barbecue Area,Basketball Court,Beauty Salon - on charge,Billiards,Board Room,Bowling,Business Center,CCTV,Campfire / Bon Fire,Car Rental,Casino,Catering,Cell Phone Rental,Children's Park,City Shuttle Service,Clubhouse,Coffee Shop / Cafe,Computer Rental,Concierge,Conference Facility,Courier Service,Currency Exchange,Disabled Friendly Facilities,Disabled Friendly Rooms,Discotheque,Doctor on Call,Doorman,Dry Cleaning,Electronic / Magnetic Keys,Electronic Check-Out,Exhibit Space,Express Check-In / Check-Out,Express Laundry,Extra Person / Child - Share the same room,Family Room,Fax Machine,Fire Place Available,Fire Safety,First-Aid Kit at Front Desk,Fishing,Fitness Equipment,Floor Butler,Flower Shop,Free Local Calls,Front Desk,Full Time Operation for All 7 Days,Fun Floats,Games Zone,Gardens,Gift Shop,Golf Course,Grocery,Guide / Sightseeing Service,Health Club / Gym Facility Available,Hookah Lounge,In House Events,Internet Access - Surcharge,Jacuzzi,Karaoke,Kids Play Zone,Laundry Service Available,Library,Lift / Elevator,Limousine Service Available,Lobby,Local Tour / Travel Desk,Locker Facility,Major Credit Cards Accepted,Marina On Site,Massage Services,Medical Assistance Available,Metal Detectors,Mini Theatre,Multi Cuisine Restaurant,Multi Lingual Staff,Nature Walk,Night Club,Open Air Restaurant / Dining,Oxygen Cylinder (on request),Paid Transfers,Parking Facility,Party hall,Pest Control Facility,Pets Allowed,"Pick Up and Drop (Rly Station, Bus Stand, Temple)",Picnic Area,Pool Table,Postal / Parcel Services,Power Backup,Private / Plunge Pool,Private Beach,Public Restrooms,RO Water Purification System,Recreation Zone,Restaurant,Rooftop Garden,Sauna,Secretarial Services,Security at All Hotel Entrances,Self Check - In,Separate Sitting Area,Skiing Facility Available,Smoke Detector,Smoking Area,Solarium,Speciality Restaurant,Squash court,Stamps and Mailing Facility,Suites,Swimming Pool,Table Tennis,Taxi Services,Tennis Court,Valet service,Vegetarian Food / Jain Food Available,Veranda,Virtual Golf,Voicemail,Wake-up Call / Service,Water Dispenser Available,Wedding Services Facility,Welcome Drinks,Wheelchair Accessibility - Room,Whirlpool Bath / Shower Cubicle,X-Ray Machine at Entrance,120 AC,220 AC,Air Conditioning,Air Cooler,Alarm Clock,Attached Bathroom,Balcony,Bathrobe,Bathrobes,Bathroom Amenities,Bathtub,Bathtub Only,Bedside Table,Bidet,Blackout Drapes,Breakfast Available,Breakfast chargeable,Ceiling Fan,Centrally Heated,Channel Music,Climate Control,Closet,Clothes Dryer,Clothes Hook in Bathroom,Coffee Table,Complete Bedding,Complimentary Use of Bicycle,Computer,Cribs Available,Crockery / Cutlery,Curtains/Shades,Daily Room Cleaning,Desk,Direct Dial Phone,Disabled Friendly Bathrooms,Do-Not-Disturb Notice,Dressing Area,Dustbins,Ensuite / Private Bathroom,"Extra Bed, Towels, Linens, Bedding (on request)",Extra Toilteries on Demand,Full Kitchen,Gas Cylinder - Chargeable,HD / LCD Satellite TV,Hairdryer,Handicapped Facilities,Hill View from all Rooms,Hot / Cold Running Water,Housekeeping,Ice Bucket on Demand,In-room Storage Space,Induction Plate,International Switch Plugs (on request),Kitchenette,Luggage Rack,Mineral Water,Mini Bar,Mini Refrigerator,Mirror,Modern Toilet Facilities,Movie Channels,Movies &amp; Games on demand,Newspapers,No Smoking Room,Patio - Property,Phone Messaging Services,Pillow menu,Premium Amenities / Toiletries,Premium Beddings / Hypo-Allergenic,Primium Bathrooms,Private Garden,Radio,Reading Lamps,Remote Control TV

In [49]:
# save user dataset
# df_ML_user.to_csv('Final_Dataset_User_ML.csv') # for ML
# df_nonML_user.to_csv('Final_Dataset_User_nonML.csv') # for non ML

## Review Data

In [51]:
df_review.dtypes

review_id               object
user_id                 object
hotel_id                object
reviews.username        object
reviews.userCity        object
reviews.userProvince    object
name                    object
reviews.rating           int64
reviews.text            object
reviews.title           object
reviews.date            object
dtype: object

In [60]:
df_review['reviews.date'] = pd.to_datetime(df_review['reviews.date'])
df_review.rename(columns={'reviews.username': 'username', 'reviews.userCity':'user_city', 'reviews.userProvince':'user_state', 'reviews.rating':'review_rating', 'reviews.text':'review_text', 'reviews.title':'review_title', 'reviews.date':'review_date', 'name':'hotel_name'}, inplace=True)
df_review

,review_id,user_id,hotel_id,username,user_city,user_state,hotel_name,review_rating,review_text,review_title,review_date
0,R000001,U000001,H000021,tatsurok2018,San Jose,UnitedStates,Best Western Plus South Coast Inn,3,"This hotel was nice and quiet. Did not know, t...",Best Western Plus Hotel,2018-01-01 00:00:00+00:00
1,R000002,U000002,H001054,STEPHEN N,San Francisco,CA,Best Western Carmel's Town House Lodge,4,We stayed in the king suite with the separatio...,Clean rooms at solid rates in the heart of Carmel,2016-04-02 00:00:00+00:00
2,R000003,U000003,H001054,15Deborah,Prescott Valley,AZ,Best Western Carmel's Town House Lodge,3,"Parking was horrible, somebody ran into my ren...",Business,2016-01-06 00:00:00+00:00
3,R000004,U000004,H001054,Wilfredo M,Guaynabo,PR,Best Western Carmel's Town House Lodge,5,Not cheap but excellent location. Price is som...,Very good,2016-08-22 00:00:00+00:00
4,R000005,U000005,H001054,Luc D,Reno,NV,Best Western Carmel's Town House Lodge,2,If you get the room that they advertised on th...,Low chance to come back here,2016-03-21 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
9782,R009783,U009215,H001298,Tiffany1017,Wallingford,CT,Hampton Inn Hampton-newport News,4,My friends and I took a trip to Hampton for th...,Very accommodating and friendly staff!,2015-12-24 00:00:00+00:00
9783,R009784,U009216,H001298,bobg187,Homer,AK,Hampton Inn Hampton-newport News,5,"from check in to departure, staff is friendly,...","comfortable, friendly, clean, professional",2015-11-17 00:00:00+00:00
9784,R009785,U009217,H001298,K261ANbrendah,Conway,AR,Hampton Inn Hampton-newport News,5,This Hampton is located on a quiet street acro...,Great location,2016-07-06 00:00:00+00:00
9785,R009786,U009218,H001103,soccerrocks2016,Hunter,NewYork,Roseberry's Inn,5,Awesome wings (my favorite was garlic parmesan...,Great Atmosphere!,2016-10-26 00:00:00+00:00


In [61]:
# save review dataset
df_review.to_csv('Final_Dataset_Review.csv') # for ML dan non ML